In [1]:
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
load_dotenv()


True

In [2]:
contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

In [3]:
print(contract_address)

0x2dbDda81e2e1C893882516B410c5Dc4e0aC027bC


In [4]:
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

In [5]:

#@st.cache_resource()
def load_contract(w3):

    # Load the contract ABI
    with open(Path('./contracts/compiled/mem_token_abi.json')) as f:
        mem_token_abi = json.load(f)

    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    # Load the contract
    contract = w3.eth.contract(
        address=contract_address,
        abi=mem_token_abi
    )

    return contract



In [6]:
contract = load_contract(w3)

In [7]:
print(contract)

In [8]:
traits = contract.functions.getMembershipInfo(0).call()

In [9]:
print(traits)

['test1', '1', 10, 0]


In [10]:
class Trait:
    def __init__(self, name, value, maxSupply, mintedCount):
        self.name = name
        self.value = value
        self.maxSupply = maxSupply
        self.mintedCount = mintedCount

In [30]:
trait_data = contract.functions.getMembershipInfo(0).call()
name, value, maxSupply, mintedCount = trait_data
trait = Trait(name, value, maxSupply, mintedCount)
print("Trait Data:")
print(f"Name: {trait.name}")
print(f"Value: {trait.value}")
print(f"Max Supply: {trait.maxSupply}")
print(f"Remaining: {trait.maxSupply - trait.mintedCount}")

Trait Data:
Name: test1
Value: 1
Max Supply: 10
Remaining: 6


In [39]:
trait_options = []

for ele in list(range(0,2)):
    trait_data = contract.functions.getMembershipInfo(ele).call()
    name, value, maxSupply, mintedCount = trait_data
    trait_form = {f'Name: 

In [40]:
trait_options

[['test1', '1', 10, 4], ['test2', '1', 10, 1]]

In [19]:
address='0xfe28b170E3F283DaA8A98ef84A09ff0A004F37F3'
acctbal = w3.eth.getBalance(address)
print(acctbal)

133941175035896432207


In [28]:

tx_hash = contract.functions.mintMembership(
address,0).transact({'from': address, 'gas': 1000000})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Transaction receipt mined:")
print(dict(receipt))
print(f"Purchase Complete!")

Transaction receipt mined:
{'transactionHash': HexBytes('0x45af0d64b75c0e85aaf9d2c69d810a4b3a4fdc289e931ca25a05cbefc470f176'), 'transactionIndex': 0, 'blockNumber': 45, 'blockHash': HexBytes('0xa142c1c233734d8d99d13c77ab3881068dfbffce04dec2d61cef4bc74a0d4611'), 'from': '0xfe28b170E3F283DaA8A98ef84A09ff0A004F37F3', 'to': '0x2dbDda81e2e1C893882516B410c5Dc4e0aC027bC', 'cumulativeGasUsed': 258886, 'gasUsed': 258886, 'contractAddress': None, 'logs': [AttributeDict({'address': '0x2dbDda81e2e1C893882516B410c5Dc4e0aC027bC', 'blockHash': HexBytes('0xa142c1c233734d8d99d13c77ab3881068dfbffce04dec2d61cef4bc74a0d4611'), 'blockNumber': 45, 'data': '0x', 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x000000000000000000000000fe28b170e3f283daa8a98ef84a09ff0a004f37f3'), HexBytes('0x00000000000000000000000000000000000000000000000000000000